### 01 packages

In [7]:
##########################################################################################
##########################################################################################

import matplotlib.pyplot as plt

from matplotlib.ticker import MultipleLocator, FormatStrFormatter

##########################################################################################
##########################################################################################

import numpy as np

import pandas as pd

##########################################################################################
##########################################################################################

import os,sys

import copy

import math

import random

import time

import datetime

from math import radians, sin, cos, sqrt, atan2

##########################################################################################
##########################################################################################

import h3

import folium

from shapely.geometry import Point, Polygon

import geopandas as gp

##########################################################################################
##########################################################################################


from scipy.stats import gamma

from scipy.stats import poisson

from scipy.stats import truncnorm

##########################################################################################
##########################################################################################

import warnings

warnings.filterwarnings("ignore")

### 02 parameters

In [8]:
##########################################################################################
##########################################################################################

# Set the lambda parameter for the Poisson distribution

lambda_value = 5

# Number of arrival times to generate

num_arrivals = 300

##########################################################################################
##########################################################################################

Hexagons=np.load('./Processed_data/Hexagons.npy')

Zone_geometry=np.load('./Processed_data/Zone_geometry.npy',allow_pickle=True).item()

Grocerys=np.load('./Processed_data/Grocerys.npy')

##########################################################################################
##########################################################################################

resolution=7

##########################################################################################
##########################################################################################

# parameters of the delivery distribution 

min_0,max_0,mu_0,sigma_0=1000,10000,4000,2000

##########################################################################################
##########################################################################################

# parameters of the earliest pick-up

min_1,max_1,mu_1,sigma_1=0.5,2.5,1.5,0.5

##########################################################################################
##########################################################################################

# parameters of the detour ratio

min_2,max_2,mu_2,sigma_2=2,8,5,3

##########################################################################################
##########################################################################################

speed=20000/60

### Synthesize

In [9]:
##########################################################################################
##########################################################################################

def generate_truncated_gaussian(lower_bound, upper_bound, mean, std, size=1):
    # Calculate the parameters of the corresponding standard normal distribution
    lower_bound_standard = (lower_bound - mean) / std
    upper_bound_standard = (upper_bound - mean) / std

    # Generate random numbers from the truncated Gaussian distribution
    random_numbers = truncnorm.rvs(lower_bound_standard, upper_bound_standard, loc=mean, scale=std, size=size)

    return random_numbers[0]

##########################################################################################
##########################################################################################

# Generate random inter-arrival times from the exponential distribution

inter_arrival_times = np.random.exponential(scale=1/lambda_value, size=num_arrivals)

# Generate the continuous arrival times by taking the cumulative sum

arrival_times = np.cumsum(inter_arrival_times)

# Round the arrival times to two decimal places

rounded_arrival_times = np.round(arrival_times, decimals=1)

# Sort the arrival times in ascending order

sorted_arrival_times = np.sort(rounded_arrival_times)

parcel_df=pd.DataFrame(sorted_arrival_times,columns=['t_a'])

parcel_df['parcel_id']=['p'+str(i+1) for i in parcel_df.index]

parcel_df=parcel_df[['parcel_id','t_a']]

##########################################################################################
##########################################################################################

pick_up_x_s,pick_up_y_s=list(),list()

for i in range(num_arrivals):
    
    # Generate a random index to select a row
    random_index = np.random.randint(0, len(Grocerys))

    # Select the randomly chosen row
    grocery = Grocerys[random_index]
    
    # Pick up Coordinates
    pick_up_x,pick_up_y=grocery[0],grocery[1]
    
    pick_up_x_s.append(pick_up_x)
    
    pick_up_y_s.append(pick_up_y)


parcel_df['pick_up_x']=pick_up_x_s

parcel_df['pick_up_y']=pick_up_y_s

parcel_df['pick_up_hexagon']=parcel_df.apply(lambda x:h3.geo_to_h3(x.pick_up_y, x.pick_up_x, resolution),axis=1)

##########################################################################################
##########################################################################################

def synthesize_dropoff(pick_up_x,pick_up_y):

    is_covered=0
    
    while is_covered==0:
        
        delivery_distance=generate_truncated_gaussian(min_0,max_0,mu_0,sigma_0)

        degree_distance=delivery_distance/111139.0

        angle = np.random.random()*np.pi*2

        drop_off_x,drop_off_y = pick_up_x + np.cos(angle)*degree_distance, pick_up_y + np.sin(angle)*degree_distance

        drop_off_hexagon=h3.geo_to_h3(drop_off_y, drop_off_x, resolution)
        
        for polygon in Zone_geometry.values():
            
            is_covered = polygon.contains(Point(drop_off_x,drop_off_y))

            if is_covered and drop_off_hexagon in Hexagons:
                
                break
            
        if is_covered and drop_off_hexagon in Hexagons:
            
            break

    return drop_off_x,drop_off_y,drop_off_hexagon
        
parcel_df['drop_off_x'], parcel_df['drop_off_y'], parcel_df['drop_off_hexagon'] = zip(*parcel_df.apply(lambda x:synthesize_dropoff(x.pick_up_x,x.pick_up_y), axis=1))

##########################################################################################
##########################################################################################

parcel_df['t_pe']=parcel_df.apply(lambda x:x.t_a+generate_truncated_gaussian(min_1,max_1,mu_1,sigma_1),axis=1)

##########################################################################################
##########################################################################################

def haversine_distance(lat1, lon1, lat2, lon2):
    
    return Point(lat1, lon1).distance(Point(lat2, lon2))*111194.92


parcel_df['shortest_travel_time']=parcel_df.apply(lambda x:haversine_distance(x.pick_up_y,x.pick_up_x,x.drop_off_y,x.drop_off_x)/speed,axis=1)

parcel_df['t_dl']=parcel_df.apply(lambda x:x.t_pe+generate_truncated_gaussian(min_2,max_2,mu_2,sigma_2)*x.shortest_travel_time,axis=1)

##########################################################################################
##########################################################################################

parcel_df.to_csv('instance1.csv')

parcel_df


,parcel_id,t_a,pick_up_x,pick_up_y,pick_up_hexagon,drop_off_x,drop_off_y,drop_off_hexagon,t_pe,shortest_travel_time,t_dl
0,p1,0.1,-74.012615,40.705974,872a10728ffffff,-73.998872,40.728885,872a1072cffffff,2.069356,8.912054,45.969745
1,p2,0.1,-73.994121,40.742090,872a100d2ffffff,-73.999469,40.731016,872a1072cffffff,1.052756,4.102407,22.330643
2,p3,0.2,-74.000095,40.738538,872a100d2ffffff,-73.999213,40.724451,872a1072cffffff,1.332784,4.708634,13.353765
3,p4,0.3,-73.953145,40.772145,872a10089ffffff,-73.965661,40.761412,872a100d6ffffff,1.888952,5.499872,19.953801
4,p5,0.4,-73.938531,40.797156,872a1008dffffff,-73.948307,40.825601,872a100aaffffff,1.304933,10.033441,64.786149
...,...,...,...,...,...,...,...,...,...,...,...
295,p296,59.1,-73.952076,40.786701,872a10089ffffff,-73.974683,40.780300,872a1008bffffff,60.688834,7.837859,116.508003
296,p297,59.3,-73.940887,40.788805,872a1008dffffff,-73.930164,40.845064,872a100aeffffff,60.868347,19.105152,196.315979
297,p298,59.5,-73.938069,40.812656,872a100abffffff,-73.968100,40.804956,872a10088ffffff,61.326015,10.341846,82.590175
298,p299,59.8,-73.963337,40.797869,872a10088ffffff,-73.969638,40.748912,872a100d0ffffff,60.509035,16.466037,120.346866


### pick-up heatmap

In [10]:
hexagon_df=pd.DataFrame(Hexagons,columns=['hexagon'])

##########################################################################################
##########################################################################################


pickup_count=parcel_df.groupby('pick_up_hexagon').count()

pickup_count=pickup_count[['parcel_id']]

pickup_count['hexagon']=pickup_count.index

pickup_count=pickup_count.reset_index(drop=True)

pickup_count.rename(columns={'parcel_id':'value'},inplace=True)

pickup_count=pickup_count[['hexagon','value']]

pickup_count=hexagon_df.merge(pickup_count,how='left',on='hexagon')

pickup_count=pickup_count.fillna(0)

##########################################################################################
##########################################################################################

latitude = 40.7831

longitude = -73.9712

##########################################################################################
##########################################################################################

network = folium.Map(location=[latitude, longitude], zoom_start=12,tiles='CartoDB positron')

hexagon_vertices = {hexagon:h3.h3_to_geo_boundary(str(hexagon)) for hexagon in Hexagons}

midpoints=np.load('./Processed_data/Lockers.npy')

for hexagon,vertices in hexagon_vertices.items():
    
    folium.Polygon(locations=vertices, 
                   color='black', 
                   opacity=1, 
                   fill=False,
                   weight=1,).add_to(network)
    
##########################################################################################
##########################################################################################

def get_hexagon_vertices(h3_index):
    return [list(reversed(coord)) for coord in h3.h3_to_geo_boundary(h3_index)]

geojson_data = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {"id": row.hexagon},
            "geometry": {
                "type": "Polygon",
                "coordinates": [get_hexagon_vertices(row.hexagon)]
            }
        }
        for idx,row in pickup_count.iterrows()
    ]
}

folium.Choropleth(
    geo_data=geojson_data,
    data=pickup_count,
    columns=["hexagon", "value"],
    key_on="feature.properties.id",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Heatmap"
).add_to(network)

    
network.save('./Html/03_pickup_hetmap.html')
    
network

### drop-off heatmap

In [11]:
hexagon_df=pd.DataFrame(Hexagons,columns=['hexagon'])

##########################################################################################
##########################################################################################


dropoff_count=parcel_df.groupby('drop_off_hexagon').count()

dropoff_count=dropoff_count[['parcel_id']]

dropoff_count['hexagon']=dropoff_count.index

dropoff_count=dropoff_count.reset_index(drop=True)

dropoff_count.rename(columns={'parcel_id':'value'},inplace=True)

dropoff_count=dropoff_count[['hexagon','value']]

dropoff_count=hexagon_df.merge(dropoff_count,how='left',on='hexagon')

dropoff_count=dropoff_count.fillna(0)

##########################################################################################
##########################################################################################

latitude = 40.7831

longitude = -73.9712

##########################################################################################
##########################################################################################

network = folium.Map(location=[latitude, longitude], zoom_start=12,tiles='CartoDB positron')

hexagon_vertices = {hexagon:h3.h3_to_geo_boundary(str(hexagon)) for hexagon in Hexagons}

midpoints=np.load('./Processed_data/Lockers.npy')

for hexagon,vertices in hexagon_vertices.items():
    
    folium.Polygon(locations=vertices, 
                   color='black', 
                   opacity=1, 
                   fill=False,
                   weight=1,).add_to(network)
    
##########################################################################################
##########################################################################################

def get_hexagon_vertices(h3_index):
    return [list(reversed(coord)) for coord in h3.h3_to_geo_boundary(h3_index)]

geojson_data = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {"id": row.hexagon},
            "geometry": {
                "type": "Polygon",
                "coordinates": [get_hexagon_vertices(row.hexagon)]
            }
        }
        for idx,row in dropoff_count.iterrows()
    ]
}

folium.Choropleth(
    geo_data=geojson_data,
    data=dropoff_count,
    columns=["hexagon", "value"],
    key_on="feature.properties.id",
    fill_color="BuGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Heatmap"
).add_to(network)

    
network.save('./Html/04_dropoff_hetmap.html')
    
network